### All days of the challange:

Here's what we're going to do today:

* [Take a first look at the data](#Take-a-first-look-at-the-data)
* [See how many missing data points we have](#See-how-many-missing-data-points-we-have)
* [Figure out why the data is missing](#Figure-out-why-the-data-is-missing)
* [Drop missing values](#Drop-missing-values)
* [Filling in missing values](#Filling-in-missing-values)

Let's get started!

# Take a first look at the data
________

The first thing we'll need to do is load in the libraries and datasets we'll be using. For today, I'll be using a dataset of events that occured in American Football games for demonstration, and you'll be using a dataset of building permits issued in San Francisco.

> **Important!** Make sure you run this cell yourself or the rest of your code won't work!

In [4]:
# libraries we'll use
import pandas as pd
# read in all our data NFL nfl1-> Play by Play 2009-2017 (v4), nfl2 -> NFL Play by Play 2009-2016 (v3)
nfl1 = pd.read_csv('NFL Play by Play 2009-2016 (v3).csv', low_memory=False)
nfl2 = pd.read_csv('NFL Play by Play 2009-2017 (v4).csv', low_memory=False)


The first thing I do when I get a new dataset is take a look at some of it. This lets me see that it all read in correctly and get an idea of what's going on with the data. In this case, I'm looking to see if I see any missing values, which will be reprsented with `NaN` or `None`.

In [8]:
# look at a few rows of the nfl1_data file. I can see a handful of missing data already!
print(nfl1.head())

         Date      GameID  Drive  qtr  down   time  TimeUnder  TimeSecs  \
0  2009-09-10  2009091000      1    1   NaN  15:00         15    3600.0   
1  2009-09-10  2009091000      1    1   1.0  14:53         15    3593.0   
2  2009-09-10  2009091000      1    1   2.0  14:16         15    3556.0   
3  2009-09-10  2009091000      1    1   3.0  13:35         14    3515.0   
4  2009-09-10  2009091000      1    1   4.0  13:27         14    3507.0   

   PlayTimeDiff SideofField  ...    yacEPA  Home_WP_pre  Away_WP_pre  \
0           0.0         TEN  ...       NaN     0.485675     0.514325   
1           7.0         PIT  ...  1.146076     0.546433     0.453567   
2          37.0         PIT  ...       NaN     0.551088     0.448912   
3          41.0         PIT  ... -5.031425     0.510793     0.489207   
4           8.0         PIT  ...       NaN     0.461217     0.538783   

   Home_WP_post  Away_WP_post  Win_Prob       WPA    airWPA    yacWPA  Season  
0      0.546433      0.453567  0.485

Yep, it looks like there's some missing values. What about in the nfl2 dataset?

In [10]:
# your turn! Look at a couple of rows from the nfl2 dataset. Do you notice any missing data?
print(nfl2.head())
# your code goes here :)

         Date      GameID  Drive  qtr  down   time  TimeUnder  TimeSecs  \
0  2009-09-10  2009091000      1    1   NaN  15:00         15    3600.0   
1  2009-09-10  2009091000      1    1   1.0  14:53         15    3593.0   
2  2009-09-10  2009091000      1    1   2.0  14:16         15    3556.0   
3  2009-09-10  2009091000      1    1   3.0  13:35         14    3515.0   
4  2009-09-10  2009091000      1    1   4.0  13:27         14    3507.0   

   PlayTimeDiff SideofField  ...    yacEPA  Home_WP_pre  Away_WP_pre  \
0           0.0         TEN  ...       NaN     0.485675     0.514325   
1           7.0         PIT  ...  1.146076     0.546433     0.453567   
2          37.0         PIT  ...       NaN     0.551088     0.448912   
3          41.0         PIT  ... -5.031425     0.510793     0.489207   
4           8.0         PIT  ...       NaN     0.461217     0.538783   

   Home_WP_post  Away_WP_post  Win_Prob       WPA    airWPA    yacWPA  Season  
0      0.546433      0.453567  0.485

# See how many missing data points we have
___

Ok, now we know that we do have some missing values. Let's see how many we have in each column. 

In [18]:
# get the number of missing data points per column
missing1 = nfl1.isnull().sum()
missing2 = nfl2.isnull().sum()

# look at the # of missing points in the first ten columns
print(missing1.head(10))
print ('---------------------------')
print(missing2.head(10))

Date                0
GameID              0
Drive               0
qtr                 0
down            54218
time              188
TimeUnder           0
TimeSecs          188
PlayTimeDiff      374
SideofField       450
dtype: int64
---------------------------
Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64


That seems like a lot! It might be helpful to see what percentage of the values in our dataset were missing to give us a better sense of the scale of this problem:

In [16]:
# how many total missing values do we have?

missing1_count = nfl1.isnull().sum().sum()
missing2_count = nfl2.isnull().sum().sum()
# percent of data that is missing
print(f"missing values nfl1: {missing1_count}")
print ('---------------------------')
print(f"missing values nfl2: {missing2_count}")

missing values nfl1: 10222931
---------------------------
missing values nfl2: 11505187


Wow, almost a quarter of the cells in this dataset are empty! In the next step, we're going to take a closer look at some of the columns with missing values and try to figure out what might be going on with them.

In [20]:
# your turn! Find out what percent of the nfl2_ dataset is missing
percentage1 = (missing1_count/nfl1.size)*100
percentage2 = (missing2_count/nfl2.size)*100

print(f"percentage of missing values nfl1: {percentage1:.3f}%")
print ('---------------------------')
print(f"percentage of missing values nfl2: {percentage2:.3f}%")

percentage of missing values nfl1: 27.652%
---------------------------
percentage of missing values nfl2: 27.667%


# Figure out why the data is missing
____
 
This is the point at which we get into the part of data science that I like to call "data intution", by which I mean "really looking at your data and trying to figure out why it is the way it is and how that will affect your analysis". It can be a frustrating part of data science, especially if you're newer to the field and don't have a lot of experience. For dealing with missing values, you'll need to use your intution to figure out why the value is missing. One of the most important question you can ask yourself to help figure this out is this:

> **Is this value missing becuase it wasn't recorded or becuase it dosen't exist?**

If a value is missing becuase it doens't exist (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it might be. These values you probalby do want to keep as NaN. On the other hand, if a value is missing becuase it wasn't recorded, then you can try to guess what it might have been based on the other values in that column and row. (This is called "imputation" and we'll learn how to do it next! :)

Let's work through an example. Looking at the number of missing values in the nfl_data dataframe, I notice that the column `TimesSec` has a lot of missing values in it: 

In [40]:
# look at the # of missing points in the first ten columns
nfl1['TimeSecs'].head(10)

0    3600.0
1    3593.0
2    3556.0
3    3515.0
4    3507.0
5    3496.0
6    3460.0
7    3431.0
8    3394.0
9    3384.0
Name: TimeSecs, dtype: float64

By looking at [the documentation](https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016), I can see that this column has information on the number of seconds left in the game when the play was made. This means that these values are probably missing because they were not recorded, rather than because they don't exist. So, it would make sense for us to try and guess what they should be rather than just leaving them as NA's.

On the other hand, there are other fields, like `PenalizedTeam` that also have lot of missing fields. In this case, though, the field is missing because if there was no penalty then it doesn't make sense to say *which* team was penalized. For this column, it would make more sense to either leave it empty or to add a third value like "neither" and use that to replace the NA's.

> **Tip:** This is a great place to read over the dataset documentation if you haven't already! If you're working with a dataset that you've gotten from another person, you can also try reaching out to them to get more information.

If you're doing very careful data analysis, this is the point at which you'd look at each column individually to figure out the best strategy for filling those missing values. For the rest of this notebook, we'll cover some "quick and dirty" techniques that can help you with missing values but will probably also end up removing some useful information or adding some noise to your data.

## Your turn!

* Look at the columns `Street Number Suffix` and `Zipcode` from the `sf_permits` datasets. Both of these contain missing values. Which, if either, of these are missing because they don't exist? Which, if either, are missing because they weren't recorded?

# Drop missing values
___

If you're in a hurry or don't have a reason to figure out why your values are missing, one option you have is to just remove any rows or columns that contain missing values. (Note: I don't generally recommend this approch for important projects! It's usually worth it to take the time to go through your data and really look at all the columns with missing values one-by-one to really get to know your dataset.)  

If you're sure you want to drop rows with missing values, pandas does have a handy function, `dropna()` to help you do this. Let's try it out on our NFL dataset!

In [24]:
# remove all the rows that contain a missing value
nfl1_no_missing_rows = nfl1.dropna()
nfl2_no_missing_rows = nfl2.dropna()
print(f"nfl1 after removing rows with missing values: {nfl1_no_missing_rows.shape}")
print(f"nfl2 after removing rows with missing values: {nfl2_no_missing_rows.shape}")


nfl1 after removing rows with missing values: (0, 102)
nfl2 after removing rows with missing values: (0, 102)


Oh dear, it looks like that's removed all our data! 😱 This is because every row in our dataset had at least one missing value. We might have better luck removing all the *columns* that have at least one missing value instead.

In [26]:
# remove all columns with at least one missing value
nfl1_no_missing_cols = nfl1.dropna(axis=1)
nfl2_no_missing_cols = nfl2.dropna(axis=1)
print(f"nfl1 after removing columns with missing values: {nfl1_no_missing_cols.shape}")
print(f"nfl2 after removing columns with missing values: {nfl1_no_missing_cols.shape}")


nfl1 after removing columns with missing values: (362447, 37)
nfl2 after removing columns with missing values: (362447, 37)


In [30]:
# just how much data did we lose?
percent_loss_nfl1 = ((nfl1.size-nfl1_no_missing_cols.size) /nfl1.size)*100
percent_loss_nfl2 = ((nfl2.size-nfl2_no_missing_cols.size) /nfl2.size)*100

print(f"Percent data loss nfl1: {percent_loss_nfl1:.3f}%\n")
print(f"Percent data loss nfl2: {percent_loss_nfl2:.3f}%")


Percent data loss nfl1: 63.725%

Percent data loss nfl2: 63.725%


We've lost quite a bit of data, but at this point we have successfully removed all the `NaN`'s from our data. 

In [34]:
# Your turn! Try removing all the rows from the nfl2 dataset that contain missing values. How many are left?
nfl2_no_missing_rows.shape[0]

0

In [36]:
# Now try removing all the columns with empty values. Now how much of your data is left?
nfl2_no_missing_cols.shape[0]

407688

# Filling in missing values automatically
_____

Another option is to try and fill in the missing values. For this next bit, I'm getting a small sub-section of the NFL data so that it will print well.

In [48]:
# get a small subset of the NFL dataset
sub_nfl1= nfl1.iloc[20:40,:10].copy()
sub_nfl2= nfl2.iloc[20:40,:10].copy()

print(sub_nfl1)
print("\n---------------------------------------------------\n")
print(sub_nfl2)

          Date      GameID  Drive  qtr  down   time  TimeUnder  TimeSecs  \
20  2009-09-10  2009091000      4    1   4.0  06:48          7    3108.0   
21  2009-09-10  2009091000      5    1   1.0  06:44          7    3104.0   
22  2009-09-10  2009091000      5    1   2.0  06:09          7    3069.0   
23  2009-09-10  2009091000      5    1   3.0  05:27          6    3027.0   
24  2009-09-10  2009091000      5    1   4.0  05:02          6    3002.0   
25  2009-09-10  2009091000      6    1   1.0  04:49          5    2989.0   
26  2009-09-10  2009091000      6    1   1.0  04:27          5    2967.0   
27  2009-09-10  2009091000      6    1   1.0  04:07          5    2947.0   
28  2009-09-10  2009091000      6    1   2.0  03:45          4    2925.0   
29  2009-09-10  2009091000      6    1   3.0  03:06          4    2886.0   
30  2009-09-10  2009091000      6    1   1.0  02:28          3    2848.0   
31  2009-09-10  2009091000      6    1   2.0  01:51          2    2811.0   
32  2009-09-

We can use the Panda's fillna() function to fill in missing values in a dataframe for us. One option we have is to specify what we want the `NaN` values to be replaced with. Here, I'm saying that I would like to replace all the `NaN` values with 0.

In [50]:
# replace all NA's with 0
print(sub_nfl1.fillna(0))

          Date      GameID  Drive  qtr  down   time  TimeUnder  TimeSecs  \
20  2009-09-10  2009091000      4    1   4.0  06:48          7    3108.0   
21  2009-09-10  2009091000      5    1   1.0  06:44          7    3104.0   
22  2009-09-10  2009091000      5    1   2.0  06:09          7    3069.0   
23  2009-09-10  2009091000      5    1   3.0  05:27          6    3027.0   
24  2009-09-10  2009091000      5    1   4.0  05:02          6    3002.0   
25  2009-09-10  2009091000      6    1   1.0  04:49          5    2989.0   
26  2009-09-10  2009091000      6    1   1.0  04:27          5    2967.0   
27  2009-09-10  2009091000      6    1   1.0  04:07          5    2947.0   
28  2009-09-10  2009091000      6    1   2.0  03:45          4    2925.0   
29  2009-09-10  2009091000      6    1   3.0  03:06          4    2886.0   
30  2009-09-10  2009091000      6    1   1.0  02:28          3    2848.0   
31  2009-09-10  2009091000      6    1   2.0  01:51          2    2811.0   
32  2009-09-

I could also be a bit more savvy and replace missing values with whatever value comes directly after it in the same column. (This makes a lot of sense for datasets where the observations have some sort of logical order to them.)

In [60]:
# replace all NA's the value that comes directly after it in the same column, 
# then replace all the reamining na's with 0
print(sub_nfl2.ffill().fillna(0))

          Date      GameID  Drive  qtr  down   time  TimeUnder  TimeSecs  \
20  2009-09-10  2009091000      4    1   4.0  06:48          7    3108.0   
21  2009-09-10  2009091000      5    1   1.0  06:44          7    3104.0   
22  2009-09-10  2009091000      5    1   2.0  06:09          7    3069.0   
23  2009-09-10  2009091000      5    1   3.0  05:27          6    3027.0   
24  2009-09-10  2009091000      5    1   4.0  05:02          6    3002.0   
25  2009-09-10  2009091000      6    1   1.0  04:49          5    2989.0   
26  2009-09-10  2009091000      6    1   1.0  04:27          5    2967.0   
27  2009-09-10  2009091000      6    1   1.0  04:07          5    2947.0   
28  2009-09-10  2009091000      6    1   2.0  03:45          4    2925.0   
29  2009-09-10  2009091000      6    1   3.0  03:06          4    2886.0   
30  2009-09-10  2009091000      6    1   1.0  02:28          3    2848.0   
31  2009-09-10  2009091000      6    1   2.0  01:51          2    2811.0   
32  2009-09-

Filling in missing values is also known as "imputation", and you can find more exercises on it [in this lesson, also linked under the "More practice!" section](https://www.kaggle.com/dansbecker/handling-missing-values). First, however, why don't you try replacing some of the missing values in the sf_permit dataset?

In [68]:
#fill with mode
cat_cols = nfl1.select_dtypes(include=['object']).columns
mode_filled_nfl1 = nfl1.copy()
for col in cat_cols :
    if mode_filled_nfl1[col].isnull().sum()>0 :
        mode_val = mode_filled_nfl1[col].mode()
        if len(mode_val)>0:
            mode_filled_nfl1[col] = mode_filled_nfl1[col].fillna(mode_val[0])
            print(f"Filled {col} with mode: {mode_val[0]}")

Filled time with mode: 15:00
Filled SideofField with mode: OAK
Filled posteam with mode: NE
Filled DefensiveTeam with mode: PHI
Filled desc with mode: Two-Minute Warning
Filled ExPointResult with mode: Made
Filled TwoPointConv with mode: Failure
Filled DefTwoPoint with mode: Failure
Filled PuntResult with mode: Clean
Filled Passer with mode: D.Brees
Filled Passer_ID with mode: 00-0020531
Filled PassOutcome with mode: Complete
Filled PassLength with mode: Short
Filled PassLocation with mode: right
Filled Interceptor with mode: A.Talib
Filled Rusher with mode: F.Gore
Filled Rusher_ID with mode: 00-0023500
Filled RunLocation with mode: right
Filled RunGap with mode: end
Filled Receiver with mode: J.Jones
Filled Receiver_ID with mode: 00-0024334
Filled ReturnResult with mode: Touchback
Filled Returner with mode: D.Sproles
Filled BlockingPlayer with mode: B.Braman
Filled Tackler1 with mode: P.Posluszny
Filled Tackler2 with mode: D.Harris
Filled FieldGoalResult with mode: Good
Filled RecFumb